In [39]:
import pandas as pd

In [40]:
df = pd.read_csv(r'C:\Users\Fernando\pyproj\my_env\SPEEDLINE sales by customer detail.csv')
print(df.head())

                  Business name        Date Transaction Type      Num  \
0  1050 ATLANTIC COLLISION INC.         NaN              NaN      NaN   
1                           NaN  04/26/2024          Invoice  12453.0   
2                           NaN  04/26/2024          Invoice  12453.0   
3                           NaN  04/26/2024          Invoice  12453.0   
4                           NaN  04/26/2024          Invoice  12453.0   

                                    Memo/Description  Qty  Sales Price  \
0                                                NaN  NaN          NaN   
1                       ALLUMINUM HUB RING 78.3-71.5  4.0          0.0   
2  XPD-704 PAKAMI 20*10 10*127/139.7 -24 GLOSS BL...  5.0        100.0   
3                   POWERHUB LT33*12.50R20 A/T 10PLY  5.0        125.0   
4            7 SPLINE LUG NUT 14X1.50X51 BK KIT 24+1  1.0         15.0   

     Amount         P.O. Number  
0       NaN                 NaN  
1    0.00    JEEP WRANGLER 2024  
2  500.00    J

In [41]:
# Initial cleaning of data

# Forward-filling business names due to the names just showing on the first line of the reports 
# (that is the nature of the format of this specific report)

df['Business name'] = df['Business name'].ffill()

# Removing rows where 'Date' and 'Transaction Type' are NaN
# I decided to do this since these are the rows that are the total rows or summary lines and I won't use them
df_clean = df.dropna(subset=['Date', 'Transaction Type'])

# Converting 'Amount' to float using astype with .loc
df_clean.loc[:, 'Amount'] = df_clean['Amount'].astype(float, errors='ignore')

# Converting 'Date' to datetime using pd.to_datetime and then astype with .loc
df_clean.loc[:, 'Date'] = pd.to_datetime(df_clean['Date'], errors='coerce').astype('datetime64[ns]')

In [42]:
df = df_clean
print(df.info())

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Noticed that some numbers in Amount column and sales price have different formats so I want to clean this data and change to float
# removing the $ signs and changing the data type

# Remove dollar signs and commas
df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True)
df['Sales Price'] = df['Sales Price'].replace('[\$,]', '', regex=True)

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Sales Price'] = pd.to_numeric(df['Sales Price'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 1 to 146
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Business name     92 non-null     object 
 1   Date              92 non-null     object 
 2   Transaction Type  92 non-null     object 
 3   Num               92 non-null     float64
 4   Memo/Description  92 non-null     object 
 5   Qty               91 non-null     float64
 6   Sales Price       91 non-null     float64
 7   Amount            92 non-null     object 
 8   P.O. Number       31 non-null     object 
dtypes: float64(3), object(6)
memory usage: 7.2+ KB
None


In [43]:
# Grouping by 'Business name' and sum the 'Amount' - sorting them after
grouped_df = df.groupby('Business name')['Amount'].sum()

sorted_df = grouped_df.sort_values(ascending=False)

print(sorted_df)


Business name
LEJEUNE TIRE& WHEELS, INC             4270.00
GENESIS TIRE LLC                      2270.00
HOT DRIVE & SOUND                     1620.00
GENERAL TIRE HIALEAH                  1615.00
VASMARTZ TIRES & AUTOPARTS INT.       1385.00
ATW TIRES WHEELS AND AUTO             1260.00
FIRST CLASS COLLISION WORKS CORP      1216.00
1050 ATLANTIC COLLISION INC.          1140.00
OCEAN TIRE SERVICES INC               1000.00
ANTONELLA 2                            960.00
TIRES WHEELS AND MORE LLC              960.00
MIAMI BEST WHEELS                      947.50
DB MOTORING GROUP INC                  900.00
CUSTOM WHEEL                           900.00
RED LINE AUTO ACCESSORIES & WHEELS     840.00
CARRERA TIRE SHOP                      650.00
DALES WHEELS                           600.00
A AND A USED TIRES AND WHEELS INC      520.00
EBAY                                   518.40
ANTONELLA TIRES LLC                    518.00
GUAYANILLA AUTO BODY CORP              480.00
WHEEL WORLD PR WHOLE

In [44]:
# Filtering for rows where any of these columns have missing values
missing_data = df[df[['Sales Price', 'Amount']].isnull().any(axis=1)]
print(missing_data)

           Business name                 Date Transaction Type      Num  \
14   ANTONELLA TIRES LLC  2024-04-25 00:00:00          Invoice  12440.0   
29         Chad Mitchell  2024-04-26 00:00:00    Sales Receipt   6134.0   
57  GENERAL TIRE HIALEAH  2024-04-25 00:00:00          Invoice  12436.0   
61  GENERAL TIRE HIALEAH  2024-04-26 00:00:00          Invoice  12443.0   

                                     Memo/Description  Qty  Sales Price  \
14                      POWERHUB LT285/55R20 A/T 8PLY  8.0       150.00   
29                                           Shipping  NaN          NaN   
57  XPD-704 PAKAMI 22*12 12*139.7/135 -44 GLOSS BL...  4.0       295.00   
61  XPD-710 TYPHOON 20*10 12*139.7/135 -24 CHROME 106  4.0       303.75   

    Amount P.O. Number  
14     NaN         NaN  
29    30.0         NaN  
57     NaN       TAHOE  
61     NaN         NaN  


In [45]:
# Notice how ANTONELLA TIRES LLC IS NOT SHOWING AN 'AMOUNT' 
# DOUBLE CHECKED ON CSV AND IT IS SHOWING AN AMOUNT WHICH HAPPENS TO BE VERY SIGNIFICANT TOO

In [46]:
# FOUND THE ERROR - DATA TYPES IN CSV ARE NOT CONSISTENT
# SOME DATA IN COLUMNS: SALES PRICE AND AMOUNT ARE NUMBER, CURRENCY OR GENERAL AND IS NOT BEING DETECTED IN DF

In [47]:
# Defining a converter function to apply when reading the CSV
# This I noticed mid cleaning process I needed to do this due to inconsistencies in data types in the csv file
# Some amounts were not being detected by the dataframe therefore not aggregating data accurately
def remove_non_numeric(value):
    if isinstance(value, str):
        # Remove currency symbols and other non-numeric characters except decimal points
        value = value.replace('$', '').replace(',', '')
    return value
# Since there has not been a lot of cleaning done yet I will just redo every step to demonstrate the difference - NOW FIXED
df = pd.read_csv(r'C:\Users\Fernando\pyproj\my_env\SPEEDLINE sales by customer detail.csv',
                 converters={'Amount': remove_non_numeric, 'Sales Price': remove_non_numeric})
print(df.head())

                  Business name        Date Transaction Type      Num  \
0  1050 ATLANTIC COLLISION INC.         NaN              NaN      NaN   
1                           NaN  04/26/2024          Invoice  12453.0   
2                           NaN  04/26/2024          Invoice  12453.0   
3                           NaN  04/26/2024          Invoice  12453.0   
4                           NaN  04/26/2024          Invoice  12453.0   

                                    Memo/Description  Qty Sales Price  \
0                                                NaN  NaN               
1                       ALLUMINUM HUB RING 78.3-71.5  4.0      0.00     
2  XPD-704 PAKAMI 20*10 10*127/139.7 -24 GLOSS BL...  5.0    100.00     
3                   POWERHUB LT33*12.50R20 A/T 10PLY  5.0    125.00     
4            7 SPLINE LUG NUT 14X1.50X51 BK KIT 24+1  1.0     15.00     

     Amount         P.O. Number  
0                           NaN  
1    0.00    JEEP WRANGLER 2024  
2  500.00    JEEP WR

In [48]:
# Initial cleaning of data - AGAIN

# Forward-filling business names due to the names just showing on the first line of the reports 
# (that is the nature of the format of this specific report)

df['Business name'] = df['Business name'].ffill()

# Removing rows where 'Date' and 'Transaction Type' are NaN
# I decided to do this since these are the rows that are the total rows or summary lines and I won't use them
df_clean = df.dropna(subset=['Date', 'Transaction Type'])

# Converting 'Amount' to float using astype with .loc
df_clean.loc[:, 'Amount'] = df_clean['Amount'].astype(float, errors='ignore')

# Converting 'Date' to datetime using pd.to_datetime and then astype with .loc
df_clean.loc[:, 'Date'] = pd.to_datetime(df_clean['Date'], errors='coerce').astype('datetime64[ns]')

print(df_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 1 to 146
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Business name     92 non-null     object 
 1   Date              92 non-null     object 
 2   Transaction Type  92 non-null     object 
 3   Num               92 non-null     float64
 4   Memo/Description  92 non-null     object 
 5   Qty               91 non-null     float64
 6   Sales Price       92 non-null     object 
 7   Amount            92 non-null     object 
 8   P.O. Number       31 non-null     object 
dtypes: float64(2), object(7)
memory usage: 7.2+ KB
None


In [49]:
# Even though it was detecting the Amoun as a Non null value it was not aggregating it

# making simpler df name for this point forward
df = df_clean

df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Noticed that some numbers in Amount column and sales price have different formats so I want to clean this data and change to float
# removing the $ signs and changing the data type

# Removing dollar signs and commas
df['Amount'] = df['Amount'].replace('[\$,]', '', regex=True)
df['Sales Price'] = df['Sales Price'].replace('[\$,]', '', regex=True)


# Changing the necessary data types for the analysis
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Sales Price'] = pd.to_numeric(df['Sales Price'], errors='coerce')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 1 to 146
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Business name     92 non-null     object 
 1   Date              92 non-null     object 
 2   Transaction Type  92 non-null     object 
 3   Num               92 non-null     float64
 4   Memo/Description  92 non-null     object 
 5   Qty               91 non-null     float64
 6   Sales Price       91 non-null     float64
 7   Amount            92 non-null     float64
 8   P.O. Number       31 non-null     object 
dtypes: float64(4), object(5)
memory usage: 7.2+ KB
None


In [50]:
# Grouping by 'Business name' and sum the 'Amount'
grouped_df = df.groupby('Business name')['Amount'].sum()

# Sorting the summed values in descending order
sorted_df = grouped_df.sort_values(ascending=False)


# PAY ATTENTION TO ANTONELLA TIRES LLC NOW - SHOWING AN EXTRA $1200 TRANSACTION
# If this data was for a longer period of time this could be detrimental to an analysis
print(sorted_df)

Business name
LEJEUNE TIRE& WHEELS, INC             4270.00
GENERAL TIRE HIALEAH                  4010.00
GENESIS TIRE LLC                      2270.00
ANTONELLA TIRES LLC                   1718.00
HOT DRIVE & SOUND                     1620.00
VASMARTZ TIRES & AUTOPARTS INT.       1385.00
ATW TIRES WHEELS AND AUTO             1260.00
FIRST CLASS COLLISION WORKS CORP      1216.00
1050 ATLANTIC COLLISION INC.          1140.00
OCEAN TIRE SERVICES INC               1000.00
ANTONELLA 2                            960.00
TIRES WHEELS AND MORE LLC              960.00
MIAMI BEST WHEELS                      947.50
DB MOTORING GROUP INC                  900.00
CUSTOM WHEEL                           900.00
RED LINE AUTO ACCESSORIES & WHEELS     840.00
CARRERA TIRE SHOP                      650.00
DALES WHEELS                           600.00
A AND A USED TIRES AND WHEELS INC      520.00
EBAY                                   518.40
GUAYANILLA AUTO BODY CORP              480.00
WHEEL WORLD PR WHOLE

In [51]:
# Checking for the missing value rows after the fix
missing_data = df[df[['Sales Price', 'Amount']].isnull().any(axis=1)]
print(missing_data)

    Business name                 Date Transaction Type     Num  \
29  Chad Mitchell  2024-04-26 00:00:00    Sales Receipt  6134.0   

   Memo/Description  Qty  Sales Price  Amount P.O. Number  
29         Shipping  NaN          NaN    30.0         NaN  


In [52]:
# CLEARLY now we only have missing data on index 29 which is Chad Mitchell
# We can see that is due to the charge he got which is shipping so there is not a price or quantity - This just gets reported as an amount

In [53]:
# VERY INTERESTING ANALYSIS HERE FOR REPEATED BUYERS
df.reset_index()
print(df.columns)
data = df

data['Date'] = pd.to_datetime(data['Date'])

# Grouping by 'Business name' and 'Date' and count unique dates
unique_sales = data.groupby('Business name').agg({'Date': pd.Series.nunique})

# Identifying repeated buyers by setting a threshold and displaying them
repeated_buyers = unique_sales[unique_sales['Date'] > 1]

print(repeated_buyers.head(5))

Index(['Business name', 'Date', 'Transaction Type', 'Num', 'Memo/Description',
       'Qty', 'Sales Price', 'Amount', 'P.O. Number'],
      dtype='object')
                                    Date
Business name                           
GENERAL TIRE HIALEAH                   3
GENESIS TIRE LLC                       2
LEJEUNE TIRE& WHEELS, INC              2
MIAMI BEST WHEELS                      2
RED LINE AUTO ACCESSORIES & WHEELS     2


In [54]:
#TOP N Customers
#This is also included in the Viz in tableau with a parameter for top N

# Aggregating sales by 'Business name'
total_sales_by_customer = data.groupby('Business name')['Amount'].sum()
sorted_sales_by_customer = total_sales_by_customer.sort_values(ascending=False)

# Selecting top N customers, for example, top 5
top_n_customers = sorted_sales_by_customer.head(5)  # Change 5 to your desired N

# Displaying the top N customers
print(top_n_customers)

Business name
LEJEUNE TIRE& WHEELS, INC    4270.0
GENERAL TIRE HIALEAH         4010.0
GENESIS TIRE LLC             2270.0
ANTONELLA TIRES LLC          1718.0
HOT DRIVE & SOUND            1620.0
Name: Amount, dtype: float64


In [55]:
# Important lesson on cleaning your data correctly
# If data was NOT cleaned correctly here, 2 of the top 5 customers of speedline wheels would have not been presented accurately
# Which when reporting to a client being precise and accurate with our information is one of the main priorities we have